In [ ]:
# Runs on Python 3.10

import numpy as np
import imageio.v2 as iio
from pathlib import Path
import traceback
from datetime import datetime

from cellpose import io, models, utils

io.logger_setup()
print("imports ok")

2026-01-06 12:27:42,063 [INFO] WRITING LOG OUTPUT TO C:\Users\caraj\.cellpose\run.log
2026-01-06 12:27:42,064 [INFO] 
cellpose version: 	4.0.8 
platform:       	win32 
python version: 	3.10.8 
torch version:  	2.9.1+cpu
imports ok


In [ ]:
# ========= USER SETTINGS =========
# check SSD folder 
in_dir = Path(r"D:/FA_Data_in_processing/Average_for_segmentation")

use_gpu = True 

# ================================

out_dir = in_dir / "cellpose_rois"
out_dir.mkdir(exist_ok=True)

# segment only 16-bit outputs- for now
files = sorted([p for p in in_dir.glob("*.tif") if "16" in p.name.lower()])

print(f"Found {len(files)} candidate tif files in: {in_dir}")
for i, p in enumerate(files[:10]):
    print(f"{i:02d}: {p.name}")
if len(files) > 10:
    print("...")

Found 11 candidate tif files in: D:\FA_Data_in_processing\Average_for_segmentation
00: avgforseg_16_Slice1_251029_Hac15_Co2_AngReihe_4.5mMKBBS_Baseline_20pM_200pM_500pM_Slice1_3_MMStack_Default.ome.tif #1-1.tif
01: avgforseg_16_Slice2_251110_Hac15_Co2_AngReihe_4.5mMKBBS_Baseline_20pM_100pM_500pM_Slice2_1h_1_MMStack_Default.ome.tif #2-1.tif
02: avgforseg_16iinv_Slice3_251030_Hac15_Co2_AngReihe_4.5mMKBBS_Baseline_80pM_200pM_500pM_Slice3_1_MMStack_Default.ome.tif #1-1.tif
03: avgforseg_16inv_Slice1_251030_Hac15_Co2_AngReihe_4.5mMKBBS_Baseline_50pM_200pM_500pM_Slice1_2_MMStack_Default.ome.tif #1-1.tif
04: avgforseg_16inv_Slice1_251113_Hac15_Co2_AngReihe_4.5mMKBBS_Baseline_20.tif
05: avgforseg_16inv_Slice2251029_Hac15_Co2_AngReihe_4.5mMKBBS_Baseline_20pM_200pM_500pM_Slice2_1_MMStack_Default.ome.tif #1-1.tif
06: avgforseg_16inv_Slice2_251030_Hac15_Co2_AngReihe_4.5mMKBBS_Baseline_50pM_200pM_500pM_Slice2_1_MMStack_Default.ome.tif #1-1.tif
07: Avgforseg_16inv_Slice2_251113_Hac15_Co2_AngReihe_4.

In [ ]:
import PIL
print(PIL.__version__)

In [ ]:
#Segmentation

model = models.CellposeModel(gpu=use_gpu)

masks_dir = out_dir / "masks_tif"
outlines_dir = out_dir / "outlines_png"
masks_dir.mkdir(exist_ok=True)
outlines_dir.mkdir(exist_ok=True)

print("Batch started:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print("GPU:", use_gpu)

for i, p in enumerate(files, start=1):
    try:
        img = iio.imread(p)
        if img.ndim != 2:
            raise ValueError(f"Expected 2D image, got shape {img.shape}")

        # normalize
        imgf = img.astype(np.float32, copy=False)
        p1, p99 = np.percentile(imgf, (1, 99))
        den = (p99 - p1)
        if den <= 1e-6:
            raise ValueError(f"Near-zero contrast image: p1={p1}, p99={p99}")
        img0n = np.clip((imgf - p1) / den, 0, 1)

        masks, flows, styles = model.eval(img0n)

        # save masks
        mask_path = masks_dir / f"{p.stem}_masks.tif"
        iio.imwrite(mask_path, masks.astype(np.uint16))

        #save rois
        roi_path = rois_dir / f"{p.stem}_rois.tif"
        io.save_rois(masks,roi_path)

        # save outlines didnt work
        #outlines = utils.outlines_list(masks)
        #overlay = utils.draw_outlines(img0n, outlines)
        #overlay_u8 = (np.clip(overlay, 0, 1) * 255).astype(np.uint8)

        #out_path = outlines_dir / f"{p.stem}_outlines.png"
        #iio.imwrite(out_path, overlay_u8)

        #print(f"[{i:02d}/{len(files)}] DONE cells={int(masks.max())} -> {p.name}")

    except Exception as e:
        print(f"[{i:02d}/{len(files)}] FAIL -> {p.name}")
        print("   ", repr(e))
        traceback.print_exc()
        continue

print("Batch finished:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

In [ ]:
#To DO

# weiterer schritt nach roi checkin in fiji:
# bilder und rois laden über roi manage, dann video dazuladen 
#aus deinterleaved und divided video mit rois :
# roi manager> more> multi measure : measure all slices, es sollte eine column pro zelle rauskommen. 

#ich würde gern zuerst alles umbenennen sodass meine dateien nummern von 1-11 haben
# dann zuerst für #4 einmal bild+roi, 
# dann video + roi, 
# dann multimeasure
